In [40]:
#### VTA--LSTM TestBench ####

import time

from pynq import allocate
import numpy as np
np.random.seed(11)



input_dmem = np.zeros(50*16, dtype=np.int8)
#for i in range(16):
input_dmem[0*16:1*16] = np.random.randint(0,10, size=16, dtype=np.int8) #x
input_dmem[1*16:2*16] = np.random.randint(0,1, size=16, dtype=np.int8)  #h1
input_dmem[2*16:3*16] = np.random.randint(0,1, size=16, dtype=np.int8)  #h2
input_dmem[3*16:4*16] = np.random.randint(0,1, size=16, dtype=np.int8)  #h3

##### Arbitrary Input Vector #####
input_dmem[0*16:1*16] = np.asarray([ 100,10,11,22,31,41,41,61,71,81,21,81,91,31,20,42 ])
####################################


weight_dmem = np.zeros(25*256,dtype=np.int8)
for i in range(24): 
    weight_dmem[i*256:(i+1)*256] = np.random.randint(0,3, size=256, dtype=np.int8) 

weight_dmem[24*256:25*256] = np.eye(16).reshape(1,256)


bias_dmem = allocate(shape=((3*4+1)*16,), dtype=np.int32)
bias_dmem = np.zeros((3*4+1)*16,dtype=np.int32)
for i in range(12):
    bias_dmem[i*16:(i+1)*16] = np.random.randint(0,4, size=16, dtype=np.int32) 

bias_dmem[12*16:13*16 ] = input_dmem[0*16:1*16].astype(np.int32)

In [41]:
#### Test Bench ####
x  = np.asarray([i for i in input_dmem[0*16:1*16]],dtype=np.int8)
h1 = np.asarray([0 for i in range(16)],dtype=np.int32)
h2 = np.asarray([0 for i in range(16)],dtype=np.int32)
h3 = np.asarray([0 for i in range(16)],dtype=np.int32)



c1=np.zeros_like(x,dtype=np.int32)
c2=np.zeros_like(x,dtype=np.int32)
c3=np.zeros_like(x,dtype=np.int32)


Wf_a = np.asarray([i for i in weight_dmem[0*256:1*256]]).reshape(16,16)
Wi_a = np.asarray([i for i in weight_dmem[1*256:2*256]]).reshape(16,16)
Wc_a = np.asarray([i for i in weight_dmem[2*256:3*256]]).reshape(16,16)
Wo_a = np.asarray([i for i in weight_dmem[3*256:4*256]]).reshape(16,16)
Uf_a = np.asarray([i for i in weight_dmem[4*256:5*256]]).reshape(16,16)
Ui_a = np.asarray([i for i in weight_dmem[5*256:6*256]]).reshape(16,16)
Uc_a = np.asarray([i for i in weight_dmem[6*256:7*256]]).reshape(16,16)
Uo_a = np.asarray([i for i in weight_dmem[7*256:8*256]]).reshape(16,16)

Wf_b = np.asarray([i for i in weight_dmem[8*256:9*256]]).reshape(16,16)
Wi_b = np.asarray([i for i in weight_dmem[9*256:10*256]]).reshape(16,16)
Wc_b = np.asarray([i for i in weight_dmem[10*256:11*256]]).reshape(16,16)
Wo_b = np.asarray([i for i in weight_dmem[11*256:12*256]]).reshape(16,16)
Uf_b = np.asarray([i for i in weight_dmem[12*256:13*256]]).reshape(16,16)
Ui_b = np.asarray([i for i in weight_dmem[13*256:14*256]]).reshape(16,16)
Uc_b = np.asarray([i for i in weight_dmem[14*256:15*256]]).reshape(16,16)
Uo_b = np.asarray([i for i in weight_dmem[15*256:16*256]]).reshape(16,16)

Wf_c = np.asarray([i for i in weight_dmem[16*256:17*256]]).reshape(16,16)
Wi_c = np.asarray([i for i in weight_dmem[17*256:18*256]]).reshape(16,16)
Wc_c = np.asarray([i for i in weight_dmem[18*256:19*256]]).reshape(16,16)
Wo_c = np.asarray([i for i in weight_dmem[19*256:20*256]]).reshape(16,16)
Uf_c = np.asarray([i for i in weight_dmem[20*256:21*256]]).reshape(16,16)
Ui_c = np.asarray([i for i in weight_dmem[21*256:22*256]]).reshape(16,16)
Uc_c = np.asarray([i for i in weight_dmem[22*256:23*256]]).reshape(16,16)
Uo_c = np.asarray([i for i in weight_dmem[23*256:24*256]]).reshape(16,16)

b1_a = np.asarray([i for i in bias_dmem[0*16:1*16]]) 
b2_a = np.asarray([i for i in bias_dmem[1*16:2*16]]) 
b3_a = np.asarray([i for i in bias_dmem[2*16:3*16]]) 
b4_a = np.asarray([i for i in bias_dmem[3*16:4*16]]) 

b1_b = np.asarray([i for i in bias_dmem[4*16:5*16]]) 
b2_b = np.asarray([i for i in bias_dmem[5*16:6*16]]) 
b3_b = np.asarray([i for i in bias_dmem[6*16:7*16]]) 
b4_b = np.asarray([i for i in bias_dmem[7*16:8*16]]) 

b1_c = np.asarray([i for i in bias_dmem[8*16:9*16]]) 
b2_c = np.asarray([i for i in bias_dmem[9*16:10*16]]) 
b3_c = np.asarray([i for i in bias_dmem[10*16:11*16]]) 
b4_c = np.asarray([i for i in bias_dmem[11*16:12*16]]) 

h1_8 = h1.astype(np.int8)
f_a = np.maximum(np.matmul(x.astype(np.int32),np.transpose(Wf_a).astype(np.int32)).astype(np.int32)+np.matmul(h1_8.astype(np.int32),np.transpose(Uf_a).astype(np.int32)).astype(np.int32) +b1_a.astype(np.int32),0).astype(np.int32)
i_a = np.maximum(np.matmul(x.astype(np.int32),np.transpose(Wi_a).astype(np.int32)).astype(np.int32)+np.matmul(h1_8.astype(np.int32),np.transpose(Ui_a).astype(np.int32)).astype(np.int32) +b2_a.astype(np.int32),0).astype(np.int32)
c_a = np.maximum(np.matmul(x.astype(np.int32),np.transpose(Wc_a).astype(np.int32)).astype(np.int32)+np.matmul(h1_8.astype(np.int32),np.transpose(Uc_a).astype(np.int32)).astype(np.int32) +b3_a.astype(np.int32),0).astype(np.int32)
o_a = np.maximum(np.matmul(x.astype(np.int32),np.transpose(Wo_a).astype(np.int32)).astype(np.int32)+np.matmul(h1_8.astype(np.int32),np.transpose(Uo_a).astype(np.int32)).astype(np.int32) +b4_a.astype(np.int32),0).astype(np.int32)


f_a_16 = f_a.astype(np.int16)
i_a_16 = i_a.astype(np.int16)
c_a_16 = c_a.astype(np.int16)
c1_16 = c1.astype(np.int16)
c1= ( ((f_a_16.astype(np.int32)*c1_16.astype(np.int32))).astype(np.int32) + (i_a_16.astype(np.int32)*c_a_16.astype(np.int32)).astype(np.int32)).astype(np.int32) 


o_a_16 = o_a.astype(np.int16)
c1_16  = np.maximum(c1.astype(np.int32),0).astype(np.int16)
h1 = (o_a_16.astype(np.int32)*c1_16.astype(np.int32)).astype(np.int32)



h1_8= h1.astype(np.int8)
h2_8= h2.astype(np.int8)
f_b = np.maximum( ((np.matmul(h1_8.astype(np.int32),np.transpose(Wf_b).astype(np.int32)).astype(np.int32)+np.matmul(h2_8.astype(np.int8),np.transpose(Uf_b)).astype(np.int32)).astype(np.int32) +b1_b.astype(np.int32)).astype(np.int32),0).astype(np.int32)
i_b = np.maximum( ((np.matmul(h1_8.astype(np.int32),np.transpose(Wi_b).astype(np.int32)).astype(np.int32)+np.matmul(h2_8.astype(np.int8),np.transpose(Ui_b)).astype(np.int32)).astype(np.int32) +b2_b.astype(np.int32)).astype(np.int32),0).astype(np.int32)
c_b = np.maximum( ((np.matmul(h1_8.astype(np.int32),np.transpose(Wc_b).astype(np.int32)).astype(np.int32)+np.matmul(h2_8.astype(np.int8),np.transpose(Uc_b)).astype(np.int32)).astype(np.int32) +b3_b.astype(np.int32)).astype(np.int32),0).astype(np.int32)
o_b = np.maximum( ((np.matmul(h1_8.astype(np.int32),np.transpose(Wo_b).astype(np.int32)).astype(np.int32)+np.matmul(h2_8.astype(np.int8),np.transpose(Uo_b)).astype(np.int32)).astype(np.int32) +b4_b.astype(np.int32)).astype(np.int32),0).astype(np.int32)



f_b_16 = f_b.astype(np.int16)
i_b_16 = i_b.astype(np.int16)
c_b_16 = c_b.astype(np.int16)
c2_16  = c2.astype(np.int16)
c2= ((f_b_16.astype(np.int32)*c2_16.astype(np.int32)).astype(np.int32)+(i_b_16.astype(np.int32)*c_b_16.astype(np.int32)).astype(np.int32)).astype(np.int32)


o_b_16 = o_b.astype(np.int16)
c2_16 = np.maximum(c2,0).astype(np.int16)
h2 = (o_b_16.astype(np.int32)*c2_16.astype(np.int32)).astype(np.int32)


h2_8 = h2.astype(np.int8)
h3_8 = h3.astype(np.int8)
f_c = np.maximum( ((np.matmul(h2_8.astype(np.int32),np.transpose(Wf_c).astype(np.int32)).astype(np.int32)+np.matmul(h3_8.astype(np.int32),np.transpose(Uf_c).astype(np.int32)).astype(np.int32)).astype(np.int32) +b1_c.astype(np.int32)).astype(np.int32),0).astype(np.int32)
i_c = np.maximum( ((np.matmul(h2_8.astype(np.int32),np.transpose(Wi_c).astype(np.int32)).astype(np.int32)+np.matmul(h3_8.astype(np.int32),np.transpose(Ui_c).astype(np.int32)).astype(np.int32)).astype(np.int32) +b2_c.astype(np.int32)).astype(np.int32),0).astype(np.int32)
c_c = np.maximum( ((np.matmul(h2_8.astype(np.int32),np.transpose(Wc_c).astype(np.int32)).astype(np.int32)+np.matmul(h3_8.astype(np.int32),np.transpose(Uc_c).astype(np.int32)).astype(np.int32)).astype(np.int32) +b3_c.astype(np.int32)).astype(np.int32),0).astype(np.int32)
o_c = np.maximum( ((np.matmul(h2_8.astype(np.int32),np.transpose(Wo_c).astype(np.int32)).astype(np.int32)+np.matmul(h3_8.astype(np.int32),np.transpose(Uo_c).astype(np.int32)).astype(np.int32)).astype(np.int32) +b4_c.astype(np.int32)).astype(np.int32),0).astype(np.int32)



f_c_16 = f_c.astype(np.int16)
c3_16  = c3.astype(np.int16)
i_c_16 = i_c.astype(np.int16)
c_c_16 = c_c.astype(np.int16)
c3= ((f_c_16.astype(np.int32)*c3_16.astype(np.int32)).astype(np.int32)+(i_c_16.astype(np.int32)*c_c_16.astype(np.int32)).astype(np.int32)).astype(np.int32)


o_c_16 = o_c.astype(np.int16)
c3_16  = np.maximum(c3,0).astype(np.int16)
h3 = (o_c_16.astype(np.int32)*c3_16.astype(np.int32)).astype(np.int32)
h_3 = h3.astype(np.int8)



print(h1.astype(np.int8))
print(h2.astype(np.int8))
print(h3.astype(np.int8))


[-120   80 -104  -46   -3  -16  -56   81 -120  124   88   24 -108  -32
   80   22]
[ 0  0 16 48  0  0  0  0  0  0  0  0  0  0  0  0]
[ -78  118    0 -110   96    0  -16  100   76  -28 -126  -46    0 -128
    9    0]
